In [2]:
import pandas as pd

KEYWORD_FILE = "keywords/keywords-prototype-21-04-22.xlsx"
DATA_VERSION = "21-04-22"

df_keywords = pd.read_excel(KEYWORD_FILE)

df_keywords = df_keywords.loc[~df_keywords["KeywordID"].isna()]
df_keywords["KeywordID"] = df_keywords["KeywordID"].astype(int)


/root/.local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
languages = set(df_keywords.columns)
languages.remove('FlagWithoutGermany')
languages.remove('KeywordID')
languages


{'BG',
 'CS',
 'DA',
 'DE',
 'EL',
 'EN',
 'ES',
 'ET',
 'FI',
 'FR',
 'HR',
 'HU',
 'IT',
 'LI',
 'LV',
 'NL',
 'PL',
 'PT',
 'RO',
 'SK',
 'SL',
 'SV'}

In [4]:
from sqlalchemy import create_engine, select, MetaData, Table, and_
from sqlalchemy.engine import URL
from io import StringIO
import csv

connection_string = URL.create(
    drivername='postgresql+psycopg2',
    username='postgres',
    password='postgres',
    host='localhost',
    port='5432',
    database='postgres')

engine = create_engine(
    connection_string)
metadata = MetaData(bind=None)


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)


def get_language_id(language):

    table = Table(
        'l_language',
        metadata,
        schema='trend',
        autoload=True,
        autoload_with=engine
    )

    stmt = select([table.columns.id]).where(
        table.columns.short == language)
    conn = engine.connect()
    return conn.execute(stmt).first()

def get_version_id(version):
    table = Table(
        'l_version',
        metadata,
        schema='trend',
        autoload=True,
        autoload_with=engine
    )

    stmt = select([table.columns.id]).where(
        table.columns.version == version)
    conn = engine.connect()
    return conn.execute(stmt).first()
# test.to_sql('l_keyword', engine, method=psql_insert_copy)


In [5]:
version_id = get_version_id(DATA_VERSION)[0]

for language in languages:
    if 'keywords' in globals():
        del keywords
    print(language)
    language_id = get_language_id(language)[0]
    keywords = df_keywords[['KeywordID', language, 'FlagWithoutGermany']].copy()
    keywords.loc[:, ('FlagWithoutGermany')] = keywords.apply(
        lambda row: 0 if row.isna()["FlagWithoutGermany"]
        else 1,
        axis=1
    ).astype('bool')
    keywords.loc[:,'language_id'] = language_id
    keywords.loc[:,'version_id'] = version_id
    keywords.rename(columns = {'KeywordID':'keyword_id', 'FlagWithoutGermany':'without_germany', language:'keyword', 'index':'id'}, inplace=True)
    keywords = keywords[['keyword_id', 'language_id', 'version_id', 'without_germany', 'keyword']]
    keywords.to_sql('l_keyword', engine, schema='trend', method=psql_insert_copy, if_exists='append', index=False)



PL
CS
ES
NL
EL
HR
FR
SV
EN
IT
DA
SK
LV
RO
LI
ET
DE
PT
HU
BG
SL
FI
